In [1]:
## CSV Files downloaded from https://www.data.gouv.fr/fr/datasets/repertoire-national-des-associations/  Fichier RNA Waldec du 01 Mars 2022
import pandas as pd
import os
import glob

In [2]:
file_location = os.getcwd() + "/rna_waldec_20220301/"
all_files = glob.glob(os.path.join(file_location, "*.csv"))

df = pd.concat((pd.read_csv(f, delimiter=";", header=0, encoding="ISO-8859-1") for f in all_files), ignore_index=True)

/tmp/ipykernel_17912/2272544456.py:4: DtypeWarning: Columns (3,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f, delimiter=";", header=0, encoding="ISO-8859-1") for f in all_files), ignore_index=True)
/tmp/ipykernel_17912/2272544456.py:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f, delimiter=";", header=0, encoding="ISO-8859-1") for f in all_files), ignore_index=True)
/tmp/ipykernel_17912/2272544456.py:4: DtypeWarning: Columns (3,18,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f, delimiter=";", header=0, encoding="ISO-8859-1") for f in all_files), ignore_index=True)
/tmp/ipykernel_17912/2272544456.py:4: DtypeWarning: Columns (1,3,18,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat((pd.read_csv(f, delimiter=";", header=0, encoding=

In [3]:
df.columns

Index(['id', 'id_ex', 'siret', 'rup_mi', 'gestion', 'date_creat', 'date_decla',
       'date_publi', 'date_disso', 'nature', 'groupement', 'titre',
       'titre_court', 'objet', 'objet_social1', 'objet_social2',
       'adrs_complement', 'adrs_numvoie', 'adrs_repetition', 'adrs_typevoie',
       'adrs_libvoie', 'adrs_distrib', 'adrs_codeinsee', 'adrs_codepostal',
       'adrs_libcommune', 'adrg_declarant', 'adrg_complemid',
       'adrg_complemgeo', 'adrg_libvoie', 'adrg_distrib', 'adrg_codepostal',
       'adrg_achemine', 'adrg_pays', 'dir_civilite', 'siteweb', 'publiweb',
       'observation', 'position', 'maj_time'],
      dtype='object')

In [4]:
def filter_cameroon(df):
     return df[df['titre'].str.contains("CAMEROUN", case=False,na=False) | df['objet'].str.contains("CAMEROUN", case=False,na=False)]

def remove_closed(df):
    return df[df["position"].str.contains("D|S") == False]

def normalize(df):
    df['titre'].str.upper()
    df['objet'].str.lower()
    df['adrs_codepostal'] = df["adrs_codepostal"].astype(int)
    df['objet_social1'] = df["objet_social1"].astype(int)
    df['objet_social2'] = df["objet_social2"].astype(int)
    df = df.fillna('') # this will avoid nan in adrs which concatenate multiple values

    return df

def select_relevant_columns(df):
    return df[["id","titre", "objet", "objet_social1","objet_social2","adrs_numvoie","adrs_typevoie","adrs_libvoie","adrs_codepostal","adrs_libcommune", "siteweb"]]

def add_column_adrs(df):
    df["adrs"] = df['adrs_numvoie'].map(str)+ " "+ df['adrs_typevoie'].map(str)+ " "+ df['adrs_libvoie'].map(str)+" "+ \
                 df['adrs_codepostal'].map(str)+" "+df['adrs_libcommune'].map(str)
    
    from postal.expand import expand_address
    df["adrs"]= df.apply(lambda row: expand_address(row["adrs"])[0], axis=1)
    return df

df2 = df.pipe(filter_cameroon) \
        .pipe(remove_closed) \
        .pipe(normalize) \
        .pipe(select_relevant_columns) \
        .pipe(add_column_adrs)

df2.sample(5)

,id,titre,objet,objet_social1,objet_social2,adrs_numvoie,adrs_typevoie,adrs_libvoie,adrs_codepostal,adrs_libcommune,siteweb,adrs
5359,W012002482,TRAIT D'UNION FRANCO CAMEROUNAIS,"favoriser le développement économique, culture...",20005,0,62,RUE,de la Ramassière,1600,Reyrieux,,62 rue de la ramassiere 1600 reyrieux
10200,W012007337,LA MAIN TENDUE (LMT),promotion des actions et expériences locales d...,20000,0,20 bis,RUE,de Lyon,1800,Meximieux,,20 bis rue de lyon 1800 meximieux
11259,W012008396,ASSOCIATION BIA-BIA (A.B.B.),promouvoir l'intérêt général des villages came...,20000,0,,,,1340,Marsonnas,,1340 marsonnas
12172,W012009309,SOLIDARITE BRESSANE CAMEROUNAISE (SBC),solidarité et entraide par l'assistance de ces...,20000,0,2,RUE,Lalande,1000,Bourg-en-Bresse,,2 rue lalande 1000 bourg-en-bresse
12436,W012009573,ASSOCIATION TREMPLINGALLE - KAMKA,faire participer l'immigration d'origine afric...,20000,0,2,BD,Irène Joliot Curie,1000,Bourg-en-Bresse,,2 boulevard irene joliot curie 1000 bourg-en-b...
...,...,...,...,...,...,...,...,...,...,...,...,...
1834654,W953011922,CAMEROONIAN BUSINESS NETWORK,rassembler les entrepreneurs et porteurs de pr...,14040,0,2,RUE,du désert aux nuages,95800,Cergy,,2 rue du desert aux nuages 95800 cergy
1875746,W9C1003716,KAMER973-ASSOCIATION DES CAMEROUNAIS DE LA GUYANE,permettre aux personnes d'origine camerounaise...,14040,0,71,RUE,Lieutenant Goinet,97300,Cayenne,,71 rue lieutenant goinet 97300 cayenne
1884141,W9R1005550,KAMER OCEAN INDIEN,"promouvoir l'amitié, la solidarité et la frate...",14040,7075,6 B,CHEM1,Romely,97419,La Possession,,6 b chem1 romely 97419 la possession
1885143,W9R1009657,SOLIDARITES CAMEROUN REUNION,"raffermir l'amitié, la solidarité la fraternit...",9000,14035,6,RUE,Pastel,97438,Sainte-Marie,,6 rue pastel 97438 sainte-marie


In [6]:
# Downloaded from https://download.geonames.org/export/zip/
region_by_postal_codes = pd.read_csv('code-postal-geonames.tsv', delimiter="\t",
                         index_col=1).to_dict()["REGION"]

dept_by_postal_codes = pd.read_csv('code-postal-geonames.tsv', delimiter="\t",
                         index_col=1).to_dict()["DEPT"]

region_by_postal_codes["97300"] = "Guyane"
dept_by_postal_codes["97300"] = "Guyane"
region_by_postal_codes["97419"] = "Réunion"
dept_by_postal_codes["97419"] = "Réunion"
region_by_postal_codes["97438"] = "Réunion"
dept_by_postal_codes["97438"] = "Réunion"
region_by_postal_codes["97600"] = "Mayotte"
dept_by_postal_codes["97600"] = "Mayotte"


waldec_csv = pd.read_csv('rna-nomenclature-waldec.csv', delimiter=";",
                         index_col=2).to_dict()["Libellé objet social parent"]
# pprint(waldec_csv)
waldec_csv[00000] = "AUTRES"
waldec_csv[19060] = "INTERVENTIONS SOCIALES"
waldec_csv[22010] = "REPRÉSENTATION, PROMOTION ET DÉFENSE D'INTÉRÊTS ÉCONOMIQUES"
waldec_csv[6035]  = "CULTURE, PRATIQUES D'ACTIVITÉS ARTISTIQUES, PRATIQUES CULTURELLES"
waldec_csv[40510] = "ACTIVITÉS RELIGIEUSES, SPIRITUELLES OU PHILOSOPHIQUES"
waldec_csv[17035] = "SANTÉ"

In [9]:
import geocoder

def add_dept_and_region(df):
    
    def get_dept_region(code_postal):
        try:
            dept = dept_by_postal_codes[str(code_postal)]
        except KeyError:
            dept = dept_by_postal_codes[[ x for x in dept_by_postal_codes.keys() if str(code_postal) in x ][0]]

        try:
            region = region_by_postal_codes[str(code_postal)]
        except KeyError:
            region = region_by_postal_codes[[ x for x in region_by_postal_codes.keys() if str(code_postal) in x ][0]]

        return {"dept": dept,"region": region}

    df["dept"] = df.apply(lambda row: get_dept_region(row["adrs_codepostal"])["dept"],axis=1)
    df["region"] = df.apply(lambda row: get_dept_region(row["adrs_codepostal"])["region"],axis=1)
    
    return df


def add_social_object_libelle(df):
    df['social_object1_libelle'] = df.apply(lambda row: waldec_csv[int(row["objet_social1"] or 0)], axis=1)
    df['social_object2_libelle'] = df.apply(lambda row: waldec_csv[int(row["objet_social2"] or 0)], axis=1)
    
    return df
    

df2 = df2.pipe(add_dept_and_region) \
         .pipe(add_social_object_libelle)

#df2["dept"] = df2.apply(lambda row: get_dept_region(row["adrs_codepostal"])["dept"],axis=1)
#df2["region"] = df2.apply(lambda row: get_dept_region(row["adrs_codepostal"])["region"],axis=1)

# get_info("W212001727")
# get_dept_region(30913)
df2.sample(5)

,id,titre,objet,objet_social1,objet_social2,adrs_numvoie,adrs_typevoie,adrs_libvoie,adrs_codepostal,adrs_libcommune,siteweb,adrs,dept,region,social_object1_libelle,social_object2_libelle
5359,W012002482,TRAIT D'UNION FRANCO CAMEROUNAIS,"favoriser le développement économique, culture...",20005,0,62,RUE,de la Ramassière,1600,Reyrieux,,62 rue de la ramassiere 1600 reyrieux,Essonne,Île-de-France,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",AUTRES
10200,W012007337,LA MAIN TENDUE (LMT),promotion des actions et expériences locales d...,20000,0,20 bis,RUE,de Lyon,1800,Meximieux,,20 bis rue de lyon 1800 meximieux,Essonne,Île-de-France,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",AUTRES
11259,W012008396,ASSOCIATION BIA-BIA (A.B.B.),promouvoir l'intérêt général des villages came...,20000,0,,,,1340,Marsonnas,,1340 marsonnas,Essonne,Île-de-France,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",AUTRES
12172,W012009309,SOLIDARITE BRESSANE CAMEROUNAISE (SBC),solidarité et entraide par l'assistance de ces...,20000,0,2,RUE,Lalande,1000,Bourg-en-Bresse,,2 rue lalande 1000 bourg-en-bresse,Essonne,Île-de-France,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",AUTRES
12436,W012009573,ASSOCIATION TREMPLINGALLE - KAMKA,faire participer l'immigration d'origine afric...,20000,0,2,BD,Irène Joliot Curie,1000,Bourg-en-Bresse,,2 boulevard irene joliot curie 1000 bourg-en-b...,Essonne,Île-de-France,"ASSOCIATIONS CARITATIVES, HUMANITAIRES, AIDE A...",AUTRES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834654,W953011922,CAMEROONIAN BUSINESS NETWORK,rassembler les entrepreneurs et porteurs de pr...,14040,0,2,RUE,du désert aux nuages,95800,Cergy,,2 rue du desert aux nuages 95800 cergy,Val-d'Oise,Île-de-France,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",AUTRES
1875746,W9C1003716,KAMER973-ASSOCIATION DES CAMEROUNAIS DE LA GUYANE,permettre aux personnes d'origine camerounaise...,14040,0,71,RUE,Lieutenant Goinet,97300,Cayenne,,71 rue lieutenant goinet 97300 cayenne,Guyane,Guyane,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...",AUTRES
1884141,W9R1005550,KAMER OCEAN INDIEN,"promouvoir l'amitié, la solidarité et la frate...",14040,7075,6 B,CHEM1,Romely,97419,La Possession,,6 b chem1 romely 97419 la possession,Réunion,Réunion,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT...","CLUBS DE LOISIRS, RELATIONS"
1885143,W9R1009657,SOLIDARITES CAMEROUN REUNION,"raffermir l'amitié, la solidarité la fraternit...",9000,14035,6,RUE,Pastel,97438,Sainte-Marie,,6 rue pastel 97438 sainte-marie,Réunion,Réunion,ACTION SOCIOCULTURELLE,"AMICALES, GROUPEMENTS AFFINITAIRES, GROUPEMENT..."


In [10]:
from geopy.geocoders import Nominatim
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import requests_cache
requests_cache.install_cache('geocode_cache')

geolocator = Nominatim(user_agent="tchoung-te.mongulu.cm")

def add_lat_lon(df):
    def geocode(adrs):
        return geolocator.geocode(adrs,country_codes="fr",timeout=None)

    df['geocode'] = df.parallel_apply(lambda row: geocode(row["adrs"]), axis=1)
    
    # df2[df2.geocode.isnull()] 417 rows n'ont pas de géocodage
    # df2[df2.adrs_libcommune.isnull()] # mais aucune n'a le nom de la commune vide donc ce sera ça qui sera utilisé
    df.loc[df.geocode.isnull(),"geocode"] = df[df.geocode.isnull()].parallel_apply(lambda row: geocode(row["adrs_libcommune"]), axis=1)
    # 4 associations restent sans geocode, on les force
    df.loc[df.id == "W251002897","geocode"] = df[df.geocode.isnull()].apply(lambda row: geocode("Besançon"), axis=1)
    df.loc[df.id == "W302012059","geocode"] = df[df.geocode.isnull()].apply(lambda row: geocode("Nimes"), axis=1)
    df.loc[df.id == "W313015063","geocode"] = df[df.geocode.isnull()].apply(lambda row: geocode("Toulouse"), axis=1)
    df.loc[df.id == "W382000558","geocode"] = df[df.geocode.isnull()].apply(lambda row: geocode("Satolas-et-Bonce"), axis=1)
    
    df['longitude'] = df['geocode'].apply(lambda x: x.longitude)
    df['latitude'] = df['geocode'].apply(lambda x: x.latitude)
    df = df.drop(columns=["objet_social1","objet_social2","geocode"], axis=1)
    
    return df


def format_libelle_for_gogocarto(df):
    # Gogocarto lit une liste de catégories sur un champ défini et considère la virgule comme le caractère de séparation
    # On a donc opté pour remplacer la virgule(",") par le slash("/")
    df["social_object1_libelle"] = df["social_object1_libelle"].apply(lambda x: x.replace(",","/"))
    df["social_object2_libelle"] = df["social_object2_libelle"].apply(lambda x: x.replace(",","/"))
    
    return df

df2 = df2.pipe(add_lat_lon) \
         .pipe(format_libelle_for_gogocarto)
df2.sample(5)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,id,titre,objet,adrs_numvoie,adrs_typevoie,adrs_libvoie,adrs_codepostal,adrs_libcommune,siteweb,adrs,dept,region,social_object1_libelle,social_object2_libelle,longitude,latitude
5359,W012002482,TRAIT D'UNION FRANCO CAMEROUNAIS,"favoriser le développement économique, culture...",62,RUE,de la Ramassière,1600,Reyrieux,,62 rue de la ramassiere 1600 reyrieux,Essonne,Île-de-France,ASSOCIATIONS CARITATIVES/ HUMANITAIRES/ AIDE A...,AUTRES,4.822173,45.936916
10200,W012007337,LA MAIN TENDUE (LMT),promotion des actions et expériences locales d...,20 bis,RUE,de Lyon,1800,Meximieux,,20 bis rue de lyon 1800 meximieux,Essonne,Île-de-France,ASSOCIATIONS CARITATIVES/ HUMANITAIRES/ AIDE A...,AUTRES,5.191291,45.902716
11259,W012008396,ASSOCIATION BIA-BIA (A.B.B.),promouvoir l'intérêt général des villages came...,,,,1340,Marsonnas,,1340 marsonnas,Essonne,Île-de-France,ASSOCIATIONS CARITATIVES/ HUMANITAIRES/ AIDE A...,AUTRES,5.071835,46.340862
12172,W012009309,SOLIDARITE BRESSANE CAMEROUNAISE (SBC),solidarité et entraide par l'assistance de ces...,2,RUE,Lalande,1000,Bourg-en-Bresse,,2 rue lalande 1000 bourg-en-bresse,Essonne,Île-de-France,ASSOCIATIONS CARITATIVES/ HUMANITAIRES/ AIDE A...,AUTRES,5.223350,46.204509
12436,W012009573,ASSOCIATION TREMPLINGALLE - KAMKA,faire participer l'immigration d'origine afric...,2,BD,Irène Joliot Curie,1000,Bourg-en-Bresse,,2 boulevard irene joliot curie 1000 bourg-en-b...,Essonne,Île-de-France,ASSOCIATIONS CARITATIVES/ HUMANITAIRES/ AIDE A...,AUTRES,5.232158,46.209581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834654,W953011922,CAMEROONIAN BUSINESS NETWORK,rassembler les entrepreneurs et porteurs de pr...,2,RUE,du désert aux nuages,95800,Cergy,,2 rue du desert aux nuages 95800 cergy,Val-d'Oise,Île-de-France,AMICALES/ GROUPEMENTS AFFINITAIRES/ GROUPEMENT...,AUTRES,2.008573,49.047606
1875746,W9C1003716,KAMER973-ASSOCIATION DES CAMEROUNAIS DE LA GUYANE,permettre aux personnes d'origine camerounaise...,71,RUE,Lieutenant Goinet,97300,Cayenne,,71 rue lieutenant goinet 97300 cayenne,Guyane,Guyane,AMICALES/ GROUPEMENTS AFFINITAIRES/ GROUPEMENT...,AUTRES,-52.327693,4.940521
1884141,W9R1005550,KAMER OCEAN INDIEN,"promouvoir l'amitié, la solidarité et la frate...",6 B,CHEM1,Romely,97419,La Possession,,6 b chem1 romely 97419 la possession,Réunion,Réunion,AMICALES/ GROUPEMENTS AFFINITAIRES/ GROUPEMENT...,CLUBS DE LOISIRS/ RELATIONS,55.336276,-20.926966
1885143,W9R1009657,SOLIDARITES CAMEROUN REUNION,"raffermir l'amitié, la solidarité la fraternit...",6,RUE,Pastel,97438,Sainte-Marie,,6 rue pastel 97438 sainte-marie,Réunion,Réunion,ACTION SOCIOCULTURELLE,AMICALES/ GROUPEMENTS AFFINITAIRES/ GROUPEMENT...,55.563840,-20.896432


In [ ]:
def remove_space_at_the_end(x: str):
    if x is not None:
        return x.strip()

def replace_double_quote(x: str):
    if x is not None:
        return x.replace("\"\"", "'")

def normalize(data: pd.DataFrame):
    text_columns = [
        "titre", "objet", "social_object1_libelle", "social_object2_libelle"
    ]
    data[text_columns] = data[text_columns].apply(
        lambda x: x.apply(remove_space_at_the_end)
    )
    data[text_columns] = data[text_columns].apply(
        lambda x: x.apply(replace_double_quote)
    )
    data["titre"] = data["titre"].apply(lambda x: x.upper())
    data["objet"] = data["objet"].apply(lambda x: x.lower())

    return data

df2 = df2.pipe(normalize)

df2.sample(5)

In [11]:
df2.to_csv("rna-real-mars-2022-new.csv")